In [1]:
import sqlite3
import time
import os
from ortools.linear_solver import pywraplp  

def solve_mTSP_branch_and_cut(instance_id, db_file):
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # Instance information
    cursor.execute("SELECT nr_cities, nr_salesmen FROM instances WHERE instance_id = ?", (instance_id,))
    instance = cursor.fetchone()
    if not instance:
        print(f"Instance {instance_id} not found.")
        conn.close()
        return

    nr_cities, nr_salesmen = instance

    # City coordinates
    cursor.execute("SELECT city_id, x, y FROM cities WHERE instance_id = ?", (instance_id,))
    cities = cursor.fetchall()

    # Create the distance matrix
    distance_matrix = [[
        ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5 for _, x2, y2 in cities
    ] for _, x1, y1 in cities]

    # Solve with Branch and Cut
    # solver = pywraplp.Solver.CreateSolver('CBC')

    solver = pywraplp.Solver.CreateSolver("GUROBI")


    solver.SetTimeLimit(20 * 1000)  # Set time limit to 20 seconds

    # Variables
    x = {}
    for i in range(nr_cities + 1):
        for j in range(nr_cities + 1):
            if i != j:
                x[i, j] = solver.BoolVar(f'x[{i},{j}]')
    u = {}
    for i in range(1, nr_cities + 1):  # Exclude the depot (city 0)
        u[i] = solver.NumVar(0, nr_cities, f'u[{i}]')

    # Constraints
    # Each city (except the depot) is visited exactly once
    for i in range(1, nr_cities + 1):
        solver.Add(solver.Sum(x[j, i] for j in range(nr_cities + 1) if j != i) == 1)
        solver.Add(solver.Sum(x[i, j] for j in range(nr_cities + 1) if j != i) == 1)

    # Each salesman starts and ends at the depot
    solver.Add(solver.Sum(x[0, j] for j in range(1, nr_cities + 1)) == nr_salesmen)
    solver.Add(solver.Sum(x[j, 0] for j in range(1, nr_cities + 1)) == nr_salesmen)

    # Subtour elimination constraints (MTZ formulation)
    for i in range(1, nr_cities + 1):
        for j in range(1, nr_cities + 1):
            if i != j:
                solver.Add(u[i] - u[j] + (nr_cities * x[i, j]) <= nr_cities - 1)

    # Objective
    solver.Minimize(solver.Sum(distance_matrix[i][j] * x[i, j] for i in range(nr_cities + 1) for j in range(nr_cities + 1) if i != j))

    # Solve
    start_time = time.time()
    status = solver.Solve()
    end_time = time.time()

    if status not in [pywraplp.Solver.OPTIMAL, pywraplp.Solver.FEASIBLE]:
        print(f"No solution found for instance {instance_id}.")
        conn.close()
        return

    # Get the total cost and time taken
    total_cost = solver.Objective().Value()
    time_taken = end_time - start_time

    # Extract routes from the solution
    routes = []
    visited = set()  # Keep track of visited cities to avoid duplicates

    for salesman_id in range(nr_salesmen):
        route = []
        current_city = 0  # Start at the depot
        while True:
            route.append(current_city)
            visited.add(current_city)
            # Find the next city for this salesman
            try:
                next_city = next(j for j in range(nr_cities + 1) if j != current_city and j not in visited and x[current_city, j].solution_value() > 0.5)
            except StopIteration:
                route.append(0)  # Return to the depot
                break  # No valid next city found
            if next_city == 0:  # Return to the depot
                route.append(next_city)
                break
            current_city = next_city
        routes.append(route)

    # Calculate distance gap 
    distances = [sum(distance_matrix[route[i]][route[i + 1]] for i in range(len(route) - 1)) for route in routes]
    distance_gap  = max(distances) - min(distances)

    # Calculate efficiency 
    epsilon = 1e-6  # offset for division by zero
    efficiency = total_cost / (time_taken + epsilon)

    # Insert results into the algorithms table
    cursor.execute("""
        INSERT OR REPLACE INTO algorithms (instance_id, strategy, total_cost, time_taken, distance_gap, efficiency)
        VALUES (?, ?, ?, ?, ?, ?)
    """, (instance_id, "Branch and Cut", total_cost, time_taken, distance_gap, efficiency))

    # Clear previous routes and insert routes into the routes table
    cursor.execute("DELETE FROM routes WHERE instance_id = ? AND strategy = ?", (instance_id, "Branch and Cut"))  
    
    for salesman_id, route in enumerate(routes):
        cursor.execute("""
            INSERT INTO routes (instance_id, strategy, salesman_id, route)
            VALUES (?, ?, ?, ?)
        """, (instance_id, "Branch and Cut", salesman_id, str(route)))

    conn.commit()
    conn.close()

    print(f"Instance {instance_id} solved using Branch and Cut.")

# Solve for each instance
def solve_mTSP_branch_and_cut_for_all_instances(db_file):
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # Select instances that have not been solved with Branch and Cut
    cursor.execute("""
        SELECT instance_id 
        FROM instances 
        WHERE instance_id NOT IN (
            SELECT instance_id 
            FROM algorithms 
            WHERE strategy = 'Branch and Cut'
        )
    """)

    # cursor.execute("""SELECT instance_id FROM instances """)
    instances = cursor.fetchall()

    if not instances:
        print("No unsolved instances found for Branch and Cut.")
        conn.close()
        return

    for (instance_id,) in instances:
        solve_mTSP_branch_and_cut(instance_id, db_file)

    conn.close()

solve_mTSP_branch_and_cut_for_all_instances(db_file="../test_mTSP.sqlite3")

load c:\Users\ignat\AppData\Local\Programs\Python\Python312\Lib\site-packages\ortools\.libs\zlib1.dll...
load c:\Users\ignat\AppData\Local\Programs\Python\Python312\Lib\site-packages\ortools\.libs\abseil_dll.dll...
load c:\Users\ignat\AppData\Local\Programs\Python\Python312\Lib\site-packages\ortools\.libs\utf8_validity.dll...
load c:\Users\ignat\AppData\Local\Programs\Python\Python312\Lib\site-packages\ortools\.libs\re2.dll...
load c:\Users\ignat\AppData\Local\Programs\Python\Python312\Lib\site-packages\ortools\.libs\libprotobuf.dll...
load c:\Users\ignat\AppData\Local\Programs\Python\Python312\Lib\site-packages\ortools\.libs\highs.dll...
load c:\Users\ignat\AppData\Local\Programs\Python\Python312\Lib\site-packages\ortools\.libs\ortools.dll...
Instance 1 solved using Branch and Cut.
Instance 2 solved using Branch and Cut.
Instance 3 solved using Branch and Cut.
Instance 4 solved using Branch and Cut.
Instance 5 solved using Branch and Cut.
Instance 6 solved using Branch and Cut.
Instan